In [20]:
#import mysql
import json
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity

## Downloand database

In [21]:
# def connect_to_database():
#     try:
#         with open('secret.txt', 'r') as file:
#             connection = mysql.connector.connect(
#                 host='localhost',
#                 database='magic_cards',
#                 user='root',
#                 password=file.readline()
#             )
#         if connection.is_connected():
#             print("Connected to MySQL database")
#             return connection
#     except mysql.connector.Error as e:
#         print(f"Error while connecting to MySQL: {e}")
#     return None

# def export_to_csv(connection, filename='magic_cards.csv'):
#     query = """
#     SELECT name, colorIdentity, keywords, supertypes
#     FROM cards
#     """
#     df = pd.read_sql(query, connection)
#     df.to_csv(filename, index=False)
#     print(f"Data exported to {filename}")



In [22]:
# Main execution
# connection = connect_to_database()
# if connection:
#     export_to_csv(connection)
#     connection.close()
# else:
#     print("Failed to connect to the database.")

## Analysis portion

In [23]:
def load_card_data(filename='cards.csv'):
    return pd.read_csv(filename)

def prepare_features(df):
    # Process colors
    color_map = {'W': 'White', 'U': 'Blue', 'B': 'Black', 'R': 'Red', 'G': 'Green'}
    df['colorIdentity'] = df['colorIdentity'].fillna('').apply(lambda x: [color_map.get(c.strip(), c.strip()) for c in x.split(',')] if isinstance(x, str) else [])
    mlb_colors = MultiLabelBinarizer()
    color_features = pd.DataFrame(
        mlb_colors.fit_transform(df['colorIdentity']),
        columns=mlb_colors.classes_,
        index=df.index
    )
    
    # Process keywords
    df['keywords'] = df['keywords'].fillna('').apply(lambda x: [kw.strip() for kw in x.split(',')] if isinstance(x, str) else [])
    mlb_keywords = MultiLabelBinarizer()
    keyword_features = pd.DataFrame(
        mlb_keywords.fit_transform(df['keywords']),
        columns=mlb_keywords.classes_,
        index=df.index
    )
    
    # Process supertypes
    df['is_legendary'] = df['supertypes'].fillna('').str.contains('Legendary')
    
    # Combine features
    features = pd.concat([color_features, keyword_features, df[['is_legendary']]], axis=1)
    
    return features, mlb_colors, mlb_keywords

def cluster_cards(features, n_clusters=10):
    kmeans = KMeans(n_clusters=n_clusters, random_state=69, n_init=10)
    return kmeans.fit_predict(features)

def find_related_cards(df, features, legendary_card_name, top_n=1000):
    legendary_card = df[df['name'] == legendary_card_name].index[0]
    legendary_features = features.iloc[legendary_card].values.reshape(1, -1)
    
    similarities = cosine_similarity(legendary_features, features)
    similar_indices = similarities[0].argsort()[::-1][1:top_n+1]
    
    return df.iloc[similar_indices]

In [24]:
df = load_card_data()
df.drop_duplicates(subset='name', keep="last")
features, mlb_colors, mlb_keywords = prepare_features(df)

# Perform clustering
clusters = cluster_cards(features)
df['cluster'] = clusters

/var/folders/6m/877x85y12tb8t246sy17j4180000gn/T/ipykernel_52956/3459460400.py:2: DtypeWarning: Columns (3,7,12,16,20,23,25,26,31,32,33,35,39,40,41,47,52,53,61,62,66,68) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(filename)


In [25]:
print("Sample of clustered cards:")
print(df[['name', 'colorIdentity', 'keywords', 'is_legendary', 'cluster']].head(5))

Sample of clustered cards:
                name colorIdentity        keywords  is_legendary  cluster
0  Ancestor's Chosen       [White]  [First strike]         False        7
1  Ancestor's Chosen       [White]  [First strike]         False        7
2     Angel of Mercy       [White]        [Flying]         False        7
3     Angel of Mercy       [White]        [Flying]         False        7
4   Angelic Blessing       [White]              []         False        1


In [26]:
# Find related cards for a legendary creature
legendary_card_name = "Reya Dawnbringer"  # An example from your data
if legendary_card_name in df['name'].values:
    related_cards = find_related_cards(df, features, legendary_card_name)
    pd.set_option('display.max_rows', None)
    print(f"\nCards related to {legendary_card_name}:")
    print(related_cards[['name', 'colorIdentity', 'keywords', 'is_legendary', 'text']])
else:
    print(f"Legendary creature '{legendary_card_name}' not found in the database.")


Cards related to Reya Dawnbringer:
                                                             name  \
13059                                  Teshar, Ancestor's Apostle   
26166                                         Zeriam, Golden Wind   
20525                                  Teshar, Ancestor's Apostle   
76342                                 Mavinda, Students' Advocate   
26242                                         Zeriam, Golden Wind   
85668                                 Mavinda, Students' Advocate   
79694                                  Linvala, Keeper of Silence   
87725                                                 Major Teroh   
76343                                 Mavinda, Students' Advocate   
49                                               Reya Dawnbringer   
48                                               Reya Dawnbringer   
26677                                           Lieutenant Kirtar   
12639                                Oyobi, Who Split the Heavens  

In [27]:
# Print some statistics
print("\nCluster Statistics:")
print(df['cluster'].value_counts())

print("\nMost common keywords:")
print(df['keywords'].explode().value_counts().head(10))

print("\nColor distribution:")
print(df['colorIdentity'].explode().value_counts())


Cluster Statistics:
cluster
0    17077
6    15611
2    13611
4    10813
3    10324
1    10252
7     6593
8     6157
9     4250
5     3004
Name: count, dtype: int64

Most common keywords:
keywords
             57285
Flying        8811
Enchant       3294
Trample       2441
Haste         1784
Vigilance     1675
Equip         1463
Mill          1427
Flash         1384
Scry          1345
Name: count, dtype: int64

Color distribution:
colorIdentity
Green    22038
Black    21896
White    21690
Blue     21666
Red      21650
         10813
Name: count, dtype: int64


In [28]:
cards = pd.read_csv('cards.csv')
cards = cards.drop_duplicates(subset='name', keep="last")
cards = cards[['name', 'colorIdentity', 'keywords', 'text', 'edhrecRank', 'manaValue', 'uuid', 'availability', 'isOnlineOnly', 'isTextless']]
print(cards.count())
with open('Keywords.json') as json_file:
    keywords = json.load(json_file)

#load dict of all keywords
keywords = pd.DataFrame({'keywords': keywords})

#split rules text into all possible permutations, then do this for each candidate and compare amount of matching options to increase support
def split_rules(rules_text):
    splits = []
    #split at first occurrence and store, then second, and so on. then iterate and do it again
    parsed_rules = rules_text.split("\\n")[-1].split(" ")

    for i in range(len(parsed_rules)):
        cut_rules = " ".join(parsed_rules[:len(parsed_rules) - i])
        for j in range(len(cut_rules.split(" "))):
            #split at last occurrence, then take the first value in split to get full string
            sliced_rules = cut_rules.split("\\n")[-1].split(" ", j)
            # for j in range(len(bruh) - i):
            splits.append(sliced_rules[-1])
    return splits

def apriori(ref_name, related_cards):
    # ref_name is L_1
    candidates = []
    rules_matrix = []

    for text in related_cards['text'].tolist():
        rules_matrix.append(split_rules(text))
        
    k = 2
    #L_k1 = related_cards[related_cards.name != ref_name]
    L_k1 = related_cards[related_cards.name == ref_name]
    current_cand = ref_name
    while L_k1.count() > 0:
        candidates = cand_gen(L_k1)
        #if len(set(rules_matrix[0]) & set(rules_matrix[i])) > 2 and len(candidates) < 62 and (related_cards['name'].tolist()[i], rules_matrix[i][0], len(set(rules_matrix[0]) & set(rules_matrix[i]))) not in candidates:
            #currently only preserving name during testing
        candidates.append((related_cards['name'].tolist()[i], rules_matrix[i][0], len(set(rules_matrix[0]) & set(rules_matrix[i]))))
            #candidates += create_candidates(related_cards['name'].tolist()[i], related_cards)
        L_k1 = related_cards[related_cards.name != ref_name]

    return candidates

def cand_gen(L_k1):
    weights = []
    for i in range(len(L_k1)):
        print(related_cards[i])


cell_value = cards.where(cards=='Reya Dawnbringer').dropna(how='all').dropna(axis=1)
# print(related_cards[['name', 'text']])
#increase support value based on shared keywords, shared terms in rules text, and triggers
pd.set_option('display.max_colwidth', None)
#print(cards.loc[cell_value.index[-1], 'text']) 
# rules = split_rules(cards.loc[cell_value.index[-1], 'text'])

candidates = apriori(legendary_card_name, related_cards)
for i in candidates: print(i)

/var/folders/6m/877x85y12tb8t246sy17j4180000gn/T/ipykernel_52956/2268911578.py:1: DtypeWarning: Columns (3,7,12,16,20,23,25,26,31,32,33,35,39,40,41,47,52,53,61,62,66,68) have mixed types. Specify dtype option on import or set low_memory=False.
  cards = pd.read_csv('cards.csv')


("Teshar, Ancestor's Apostle", 'Whenever you cast a historic spell, return target creature card with mana value 3 or less from your graveyard to the battlefield. (Artifacts, legendaries, and Sagas are historic.)', 406)
('Zeriam, Golden Wind', 'Whenever a Griffin you control deals combat damage to a player, create a 2/2 white Griffin creature token with flying.', 6)
("Mavinda, Students' Advocate", "{0}: You may cast target instant or sorcery card from your graveyard this turn. If that spell doesn't target a creature you control, it costs {8} more to cast this way. If that spell would be put into your graveyard, exile it instead. Activate only once each turn. (You still pay the spell's costs. Timing rules for the spell still apply.)", 15)
('Reya Dawnbringer', 'At the beginning of your upkeep, you may return target creature card from your graveyard to the battlefield.', 32)
('Oyobi, Who Split the Heavens', 'Whenever you cast a Spirit or Arcane spell, create a 3/3 white Spirit creature tok